In [ ]:
import sys
print("python 版本检测",sys.version)

In [ ]:
#gpu检测
import torch
print(f"PyTorch version: {torch.__version__}")
!nvidia-smi
print("torch可使用的GPU数量是",torch.cuda.device_count())

In [ ]:
# %pip install psutil gputil

In [ ]:
import psutil
import GPUtil
import ipywidgets as widgets
from IPython.display import display
import time
import threading

# 监控线程的终止标志
stop_thread = threading.Event()

cpu_label = widgets.Label()
gpu_label = widgets.Label()

def update_metrics():
    while not stop_thread.is_set():
        # Update CPU usage
        cpu_usage = psutil.cpu_percent(interval=1)
        cpu_label.value = f"CPU 使用率: {cpu_usage}%"
        
        # Update GPU usage
        gpus = GPUtil.getGPUs()
        gpu_info = ""
        for gpu in gpus:
            gpu_info += (f"GPU {gpu.id}: GPU使用率: {gpu.load * 100}%, "
                         f"显存总量: {gpu.memoryTotal}MB, "
                         f"已用显存: {gpu.memoryUsed}MB, "
                         f"可用显存: {gpu.memoryFree}MB\n")
        gpu_label.value = gpu_info
        
        # 使用较短的 sleep 间隔来检查终止标志
        for _ in range(10):
            if stop_thread.is_set():
                break
            time.sleep(0.1)

# 启动监控线程
thread = threading.Thread(target=update_metrics)
thread.start()
display(cpu_label)
display(gpu_label)
stop_button = widgets.Button(description="停止监控")
display(stop_button)
def stop_monitoring(b):
    stop_thread.set()
    thread.join()  # 等待线程结束
    print("监控已停止")

stop_button.on_click(stop_monitoring)